In [10]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
url = 'https://github.com/matisolica/Proyecto-final-DS1/raw/bd6cb31c2509e8c662132e43d728f7f5bcd364f5/Matias%20Isolica%20Dataset%20-%20Ausentismo%20y%20coberturas%20Enfermeria%202023.xlsx'
dbexcel = pd.read_excel(url, sheet_name=None)

In [11]:
#printeo los nombres de las hojas del xlsx
print(dbexcel.keys())

dict_keys(['Tb_Coberturas', 'Tb_Ausentismo', 'Tb_Nomina', 'Tb_Motivos', 'Tb_FormaDeCobertura', 'Tb_Turnos', 'Tb_Servicios', 'Tb_Categorias', 'Tb_Localidades'])


In [14]:
#asigno nombre de variable a cada hoja del xlsx
df_cob = dbexcel['Tb_Coberturas']
df_aus = dbexcel['Tb_Ausentismo']
df_nom = dbexcel['Tb_Nomina']
df_mot = dbexcel['Tb_Motivos']
df_formcob = dbexcel['Tb_FormaDeCobertura']
df_tur = dbexcel['Tb_Turnos']
db_ser = dbexcel['Tb_Servicios']
db_cat = dbexcel['Tb_Categorias']
db_loc = dbexcel['Tb_Localidades']